# LangChain Expression Language (LCEL)

The **L**ang**C**hain **E**xpression **L**anguage (LCEL) abstracts key Python concepts into a streamlined format, facilitating a "minimalist" code layer for constructing chains of LangChain components. LCEL offers robust support for:

1. Rapid development of chains.
2. Advanced features like streaming, asynchronous processing, parallel execution, and more.
3. Seamless integration with LangSmith and LangServe.

In [24]:
# !pip install -qU \
#     langchain==0.0.345 \
#     anthropic==0.7.7 \
#     cohere==4.37 \
#     docarray==0.39.1

## LCEL Syntax

In [1]:
from ast import mod
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import ChatOllama
from rich import print as pp

prompt = ChatPromptTemplate.from_template(
    "Give me video ideas for the topic of {topic}"
)

model = ChatOllama(model="llama3.2")


output_parser = StrOutputParser()

IIn a standard LangChain setup, we would connect these components using an LLMChain:

In [2]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=output_parser
)

# and run
out = chain.run(topic="Artificial Intelligence")
print(out)

/var/folders/m7/yjmck8kn59gc9w3kdklj2lt40000gn/T/ipykernel_99256/4093089756.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
/var/folders/m7/yjmck8kn59gc9w3kdklj2lt40000gn/T/ipykernel_99256/4093089756.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  out = chain.run(topic="Artificial Intelligence")


Here are some video ideas for the topic of Artificial Intelligence:

**Explainer Videos**

1. "What is AI?" - A beginner's guide to understanding artificial intelligence.
2. "How Machine Learning Works" - An in-depth explanation of machine learning algorithms and their applications.
3. "The History of AI" - A timeline of major milestones in the development of artificial intelligence.

**Technology Showcase**

1. "10 Emerging AI Technologies You Need to Know" - A showcase of cutting-edge AI technologies, including computer vision, natural language processing, and robotics.
2. "AI-Powered Automation: How Robots Are Changing Industries" - A video highlighting the role of robots in various industries, such as manufacturing and logistics.
3. "The Future of Virtual Assistants: What You Need to Know" - An overview of current virtual assistants like Siri, Alexa, and Google Assistant.

**Success Stories**

1. "How AI Is Revolutionizing Healthcare" - A case study on how artificial intelligence i

Using LCEL the format is different, rather than relying on `Chains` we simple chain together each component using the pipe operator `|`:

In [3]:
lcel_chain = prompt | model | output_parser

# and run
out = lcel_chain.invoke({"topic": "Artificial Intelligence"})
print(out)

Here are some video ideas on the topic of Artificial Intelligence:

**Explainer Videos**

1. "What is Artificial Intelligence?" - A beginner's guide to AI, explaining its basics and how it works.
2. "How AI Works" - A step-by-step explanation of the AI process, from data collection to decision-making.
3. "The Difference between Machine Learning and Deep Learning" - A comparison of these two popular AI subfields.

**Case Study Videos**

1. "AI in Healthcare: Success Stories and Challenges" - Exploring how AI is being used in healthcare, including examples of successful applications and areas for improvement.
2. "AI-Powered Chatbots: The Future of Customer Service" - Showcasing the benefits and limitations of chatbot technology using AI.
3. "How AI is Revolutionizing Transportation" - Examining how AI is improving safety, efficiency, and sustainability in transportation.

**Interviews and Expert Insights**

1. "AI Industry Leaders Share Their Insights" - Interviewing experts from various

## How the Pipe Operator Works

To truly grasp LCEL, let's examine how the pipe operation functions. It takes output from the **right** and feeds it to the **left**—but since this isn't standard Python, how is it implemented? We can create our own version using simple functions.

We'll utilize the `__or__` method in Python class objects. When we combine two classes like `chain = class_a | class_b`, the Python interpreter checks for the presence of the `__or__` method in these classes. If it exists, the expression `|` is translated to `chain = class_a.__or__(class_b)`. 

This means both of the following patterns yield the same result:

```python
# Object approach
chain = class_a.__or__(class_b)
chain("some input")

# Pipe approach
chain = class_a | class_b
chain("some input")
```

With this understanding, we can create a `Runnable` class that takes a function and transforms it into a chainable function using the pipe operator `|`.



In [28]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

Let's implement this to take the value `3`, add `5` (giving `8`), and multiply by `2` (giving `16`).

In [29]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

chain = add_five.__or__(multiply_by_two)
chain(3)  

16

Using `__or__` directly we get the correct answer, now let's try using the pipe operator `|` to chain them together:

In [30]:
chain = add_five | multiply_by_two

chain(3)  

16

Using either method we get the same response, at it's core this is what LCEL is doing, but there is more.

## LCEL Deep Dive

Now that we understand what this syntax is doing under the hood, let's explore it within the context of LCEL and see a few of the additional methods that LangChain has provided to maximize flexibility when working with LCEL.

### Runnables

When working with LCEL we may find that we need to modify the structure or values being passed between components — for this we can use _runnables_. Let's try:

In [7]:
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Create an embedding model
embedding = OllamaEmbeddings(model="snowflake-arctic-embed:33m")

# Create two vector stores with different texts
vecstore_a = InMemoryVectorStore.from_texts(
    ["The capital of France is Paris", "The Eiffel Tower is in Paris"],
    embedding=embedding
)
vecstore_b = InMemoryVectorStore.from_texts(
    ["The Louvre is the world's largest art museum", "Paris is known for its cafe culture"],
    embedding=embedding
)

### Creating retrievers 

In [ ]:
import re
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

prompt_str =""" Answer the following questions about Paris:

Context: {context}

Question: {question}

Asnwer: """


prompt = ChatPromptTemplate.from_template(prompt_str)

retriever = RunnableParallel(
    {'context': retriever_a, 'question': RunnablePassthrough()},
)

chain = retrieval | prompt | model | output_parser

pp(chain)

In [15]:
out = chain.invoke("Where is Paris located?")
print(out)

Paris is located in France.


Here we have used `RunnableParallel` to create two parallel streams of information, one for `"context"` that is information fed in from `retriever_a`, and another for `"question"` which is the _passthrough_ information, ie the information that is passed through from our `chain.invoke("when was James born?")` call.

Using this information the chain is close to answering the question but it doesn't have enough information, it is missing the information that we have stored in `retriever_b`. Fortunately, we can have multiple parallel information streams using the `RunnableParallel` object.

In [13]:
prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | model | output_parser

In [16]:
out = chain.invoke("What is Paris known for?")
print(out)

Based on the context, Paris is known for its cafe culture.


## Runnable Lambdas
The `RunnableLambda` is a LangChain abstraction that allows us to turn Python functions into pipe-compatible function, similar to the `Runnable` class we created near the beginning of this notebook.

Let's try it out with our earlier `add_five` and `multiply_by_two` functions.

In [21]:
def calc_root(x):
    return x ** 0.5

def calc_square(x):
    return x ** 2


In [22]:
from langchain_core.runnables import RunnableLambda

# wrap the functions with RunnableLambda
add_five = RunnableLambda(calc_root)
multiply_by_two = RunnableLambda(calc_square)

As with our earlier `Runnable` abstraction, we can use `|` operators to chain together our `RunnableLambda` abstractions.

In [24]:
chain = add_five | multiply_by_two
pp(chain)

RunnableSequence(first=RunnableLambda(calc_root), middle=[], last=RunnableLambda(calc_square))

Unlike our `Runnable` abstraction, we cannot run the `RunnableLambda` chain by calling it directly, instead we must call `chain.invoke`:

In [25]:
chain.invoke(3)

2.9999999999999996

As before, we can see the same answer. Naturally we can feed custom functions into our chains using this approach. Let's try a short chain and see where we might want to insert a custom function:

In [28]:
prompt_str = "Tell me an short fact about {topic}"
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | model | output_parser
pp(chain)

RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['topic'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['topic'],
                    input_types={},
                    partial_variables={},
                    template='Tell me an short fact about {topic}'
                ),
                additional_kwargs={}
            )
        ]
    ),
    middle=[ChatOllama(model='llama3.2')],
    last=StrOutputParser()
)

In [31]:
pp(chain.invoke({"topic": "Artificial Intelligence"}))

Here's a short fact about Artificial Intelligence (AI):

The first AI program, called Logical Theorist, was developed in 1956 by Alan Newell and Herbert Simon. It was able 
to reason and solve problems using logical deduction, marking the beginning of the field of Artificial Intelligence
research.

In [32]:
pp(chain.invoke({"topic": "Artificial Intelligence"}))

Artificial Intelligence (AI) was first proposed by Alan Turing in his 1950 paper "Computing Machinery and 
Intelligence," where he posed the question, "Can machines think?" This idea laid the foundation for modern AI 
research and development.

The returned text always includes the initial `"Here's a short fact about ...\n\n"` — let's add a function to split on double newlines `"\n\n"` and only return the fact itself.

In [33]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x
    
get_fact = RunnableLambda(extract_fact)

chain = prompt | model | output_parser | get_fact
pp(chain)

RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['topic'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['topic'],
                    input_types={},
                    partial_variables={},
                    template='Tell me an short fact about {topic}'
                ),
                additional_kwargs={}
            )
        ]
    ),
    middle=[ChatOllama(model='llama3.2'), StrOutputParser()],
    last=RunnableLambda(extract_fact)
)

In [37]:
pp(chain.invoke({"topic": "Artificial Intelligence"}))

The term "Artificial Intelligence" (AI) was first coined in 1956 by John McCarthy, who used it to describe the goal
of creating machines that could simulate human intelligence and think like humans.

In [38]:
pp(chain.invoke({"topic": "Artificial Intelligence"}))

Did you know that the first AI program, called "Logical Theorist," was developed in 1956 by Allen Newell and 
Herbert Simon? This groundbreaking program could reason and solve problems using logical rules, marking the 
beginning of the field of Artificial Intelligence as we know it today.

Now we're getting well formatted outputs using our final custom `get_fact` function.

---

That covers the essentials you need to getting started and building with the **L**ang**C**hain **E**xpression **L**anguage (LCEL). With it, we can put together chains very easily — and the current focus of the LangChain team is on further LCEL development and support.

The pros and cons of LCEL are varied. Those that love it tend to focus on the minimalist code style, LCEL's support for streaming, parallel operations, and async, and LCEL's nice integration with LangChain's focus on chaining components together.

There are also people that are less fond of LCEL. Typically people will point to it being yet another abstraction on top of an already very abstract library, that the syntax is confusing, against [the Zen of Python](https://peps.python.org/pep-0020/), and requires too much effort to learn new (or uncommon) syntax.

Both viewpoints are entirely valid, LCEL is a very different approach — ofcourse there are major pros and major cons. In either case, if you're willing to spend some time learning the syntax, it allows us to develop very quickly, and with that in mind it's well worth learning.

---